In [1]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 1. Configuration MLflow
# Cela va créer un dossier 'mlruns' à la racine de l'endroit où vous lancez le notebook, c'est là que les dossiers de log seront ajoutés
mlflow.set_tracking_uri("file:../mlruns")
#On set le nom de notre "experiment", c'est en gros un dossier qui vas compiler toutes les logs de no run
mlflow.set_experiment("Credit_Scoring_Baseline")
mlflow.sklearn.autolog() #autolog pour log toutes nos metrics, il faudra aussi ajouter des log manuel pour les métriques personalisées

# 2. Chargement "brut" (Juste pour tester le pipeline)
# Path absolu du csv d'entrainement vierge sans aucune modifications pour faire des tests
PATH = r'C:\Users\mathi\OneDrive\Bureau\ESAIP\ING4\MAJEUR\Data Analysis & Engineering\Projet_Final\datasets\application_train.csv'
df = pd.read_csv(PATH, nrows=1000) # Juste pour prendre un échantillon des 1000 premières lignes
features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY'] #Quelques colonnes sinon c'est trop long 
df = df.dropna(subset=features) # Nettoyage express pour le test

X = df[features]
y = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


c:\Users\mathi\OneDrive\Bureau\ESAIP\ING4\MAJEUR\Repos\DataAnalisis\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


In [17]:
mlflow.set_experiment("MNIST_CNN_Test")

# Activez l'autolog pour Tensorflow/Keras
# Il va capturer : métriques par époque, architecture du modèle, optimizer, et le modèle lui-même.
mlflow.tensorflow.autolog(log_models=True, log_input_examples=True, log_model_signatures=True)

# 2. Préparation des données (Standard)
# --------------------------------------
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape & Normalisation
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 3. Lancement de l'entraînement avec MLflow
# ------------------------------------------
print("Lancement du run MLflow...")

with mlflow.start_run(run_name="CNN_Autolog_Run"):
    
    # A. Définition du modèle (Votre architecture CNN)
    model = Sequential([
        Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        BatchNormalization(),
        Dropout(0.25),
        
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        BatchNormalization(),
        Dropout(0.3),
        
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    
    # B. Compilation
    # Note: MLflow loggera automatiquement 'learning_rate' et 'optimizer_name'
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # C. Entraînement
    # C'est ici que la magie opère : MLflow crée des graphiques pour 'loss' et 'val_accuracy'
    history = model.fit(
        x_train, y_train,
        batch_size=128,
        epochs=5,  # Réduit pour le test
        validation_data=(x_test, y_test),
        verbose=1
    )
    
    # D. Logging MANUEL supplémentaire (Ce que autolog ne fait pas toujours bien)
    # Par exemple, une matrice de confusion ou une image test
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    mlflow.log_metric("final_test_accuracy", test_acc)
    
    print(f"Entraînement terminé. Accuracy: {test_acc:.4f}")
    print("Vérifiez l'interface MLflow (mlflow ui) pour voir les courbes d'apprentissage !")

Lancement du run MLflow...


c:\Users\mathi\OneDrive\Bureau\ESAIP\ING4\MAJEUR\Repos\DataAnalisis\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8945 - loss: 0.3637

469/469 ━━━━━━━━━━━━━━━━━━━━ 31s 62ms/step - accuracy: 0.9495 - loss: 0.1707 - val_accuracy: 0.9351 - val_loss: 0.2160
Epoch 2/5
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9811 - loss: 0.0595

469/469 ━━━━━━━━━━━━━━━━━━━━ 29s 62ms/step - accuracy: 0.9820 - loss: 0.0591 - val_accuracy: 0.9904 - val_loss: 0.0275
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 29s 62ms/step - accuracy: 0.9860 - loss: 0.0449 - val_accuracy: 0.9912 - val_loss: 0.0279
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 29s 62ms/step - accuracy: 0.9887 - loss: 0.0362 - val_accuracy: 0.9912 - val_loss: 0.0306
Epoch 5/5
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9900 - loss: 0.0317

469/469 ━━━━━━━━━━━━━━━━━━━━ 29s 62ms/step - accuracy: 0.9894 - loss: 0.0327 - val_accuracy: 0.9933 - val_loss: 0.0199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


2025/12/01 20:16:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 20:16:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\mathi\OneDrive\Bureau\ESAIP\ING4\MAJEUR\Repos\DataAnalisis\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 20 variables whereas the saved optimizer has 38 variables. "


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Entraînement terminé. Accuracy: 0.9933
Vérifiez l'interface MLflow (mlflow ui) pour voir les courbes d'apprentissage !
